In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

In [33]:
df = pd.read_csv("prep_churn_modelling.csv")
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [34]:
from sklearn.model_selection import train_test_split
X = df.drop(["Exited"],axis=1)
y = df["Exited"]

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [36]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [7]:
with open("scale.pkl","wb") as file:
    pickle.dump(scale,file)

ANN

In [37]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.models import Sequential
import datetime

In [7]:
type((X_train.shape[1],))

tuple

In [38]:
# Creating the layers of our model

model = Sequential([
    Dense(64,activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(4,activation='relu'),
    Dense(2,activation='relu'),
    Dense(1,activation='sigmoid')
])

c:\Users\prasa\OneDrive\Desktop\machine learning\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,625 (14.16 KB)

 Trainable params: 3,625 (14.16 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
import tensorflow as tf
optim = tf.keras.optimizers.Adam(learning_rate=0.01)
losses = tf.keras.losses.BinaryCrossentropy()

In [41]:
#Compiling our model

model.compile(optimizer = optim, loss = losses, metrics=['accuracy'])

In [50]:
#Creating the Tensorboard

log_direc = "log/fit/" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
tensorflow_callback = TensorBoard(log_dir = log_direc, histogram_freq = 1)

In [43]:
#Creating Early Stopping

early_stop_callback = EarlyStopping(monitor = "val_loss", patience=30, restore_best_weights=True)

In [51]:
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100, callbacks=[tensorflow_callback, early_stop_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7981 - loss: 0.4372 - val_accuracy: 0.8500 - val_loss: 0.3900
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8468 - loss: 0.3812 - val_accuracy: 0.8455 - val_loss: 0.3814
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8533 - loss: 0.3671 - val_accuracy: 0.8615 - val_loss: 0.3570
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8369 - loss: 0.3794 - val_accuracy: 0.8560 - val_loss: 0.3562
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8585 - loss: 0.3587 - val_accuracy: 0.8605 - val_loss: 0.3575
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8560 - loss: 0.3589 - val_accuracy: 0.8590 - val_loss: 0.3490
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8625 - loss: 0.3481 - val_accuracy: 0.8575 - val_loss: 0.3461
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8640 - loss: 0.3459 - val_accu

In [15]:
model.save('model.h5')

In [ ]:
#Loading Tensorboard Extension

%load_ext tensorboard

In [28]:
%tensorboard --logdir log/fit --port 6006

Reusing TensorBoard on port 6006 (pid 19112), started 0:04:24 ago. (Use '!kill 19112' to kill it.)

In [52]:
import os,signal
pid = 19112
os.kill(pid, signal.SIGTERM)

OSError: [WinError 87] The parameter is incorrect